In [1]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

# 父文档检索器
在分割文档进行检索时，经常会有相互冲突的需求:
1. 您可能希望使用较小的文档，以便它们的嵌入可以最准确地反映其含义。如果时间过长，嵌入内容就会失去意义。
2. 您希望拥有足够长的文档，以便保留每个块的上下文。

`ParentDocumentRetriever`通过分割和存储小块数据来实现这种平衡。在检索期间，它首先获取小块，然后查找这些块的父id并返回那些较大的文档。
注意，“父文档”指的是小块源自的文档。这可以是整个原始文档，也可以是更大的块。

In [2]:
from langchain.retrievers import ParentDocumentRetriever

In [3]:
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [5]:
loaders = [
    TextLoader("../../data/2015北京年报.txt",encoding='utf-8'),
    TextLoader("../../data/2016北京年报.txt",encoding='utf-8'),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

## 检索完整文档
在这种模式下，我们希望检索完整的文档。因此，我们只指定一个子拆分器。

In [6]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [7]:
retriever.add_documents(docs, ids=None)

这应该产生两个键，因为我们添加了两个文档。

In [8]:
list(store.yield_keys())

['fba6ad33-3a78-4359-9b2c-1f83b36bf651',
 '510ac612-e694-4f5e-9d0e-eef2a328e5ef']

现在我们调用向量存储搜索功能——我们应该看到它返回小块(因为我们存储的是小块)。

In [9]:
sub_docs = vectorstore.similarity_search("2015年的购买国债")

In [10]:
print(sub_docs[0].page_content)

（四）购买国债
2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。

（五）资金存储
截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。

（六）资金运用率
2015年住房公积金提取、贷款业务量大幅增加，结余资金大幅减少，资金使用效率进一步提升。截至2015年底，资金运用率（住房公积金个人住房贷款余额、项目贷款余额和购买国债余额的总和与缴存余额的比率）89.6%，比上年同期增加9.9个百分点。


现在让我们从整个检索器中检索。这将返回较大的文档——因为它返回较小块所在的文档。

In [11]:
retrieved_docs = retriever.get_relevant_documents("2015年的购买国债")

In [12]:
len(retrieved_docs[0].page_content)

5638

## 检索更大的块
有时，完整的文档可能太大而不想原样检索它们。在这种情况下，我们真正想做的是首先将原始文档分成较大的块，然后再分成较小的块。然后我们索引较小的块，但在检索时，我们检索较大的块(但仍然不是完整的文档)。

In [13]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()

In [14]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [15]:
retriever.add_documents(docs)

我们可以看到，现在有两个以上的文档-这些是较大的块。

In [16]:
len(list(store.yield_keys()))

8

让我们确保底层向量存储仍然检索小块。

In [17]:
sub_docs = vectorstore.similarity_search("2015年的购买国债")

In [18]:
print(sub_docs[0].page_content)

（四）购买国债
2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。

（五）资金存储
截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。

（六）资金运用率
2015年住房公积金提取、贷款业务量大幅增加，结余资金大幅减少，资金使用效率进一步提升。截至2015年底，资金运用率（住房公积金个人住房贷款余额、项目贷款余额和购买国债余额的总和与缴存余额的比率）89.6%，比上年同期增加9.9个百分点。


In [19]:
retrieved_docs = retriever.get_relevant_documents("2015年的购买国债")

In [20]:
len(retrieved_docs[0].page_content)

1901

In [21]:
print(retrieved_docs[0].page_content)

（三）委托贷款
1.住房公积金个人贷款
2015年，管理中心积极落实住房城乡建设部、财政部、中国人民银行相关文件精神，制定出台多项住房公积金个人贷款惠民政策，最大限度地满足职工基本住房需求，帮助广大缴存职工实现住房梦。北京地区全年发放住房公积金个人贷款848.86亿元，涉及住房101367套，回收贷款228.98亿元，同比增长175.6%、105.6%、54.0%，贷款支持住房套数、发放金额、回收金额均创历史新高。截至2015年底，北京地区累计发放住房公积金个人贷款3918.13亿元，涉及住房85.55万套，在贷住房46.50万套，贷款余额2491.69亿元，同比增长33.1%。
2015年，北京地区发放异地购房贷款4740万元，涉及住房54套；发放贴息贷款14125万元，涉及住房152套，贴息1091.83万元。累计发放贴息贷款47.3亿元，涉及住房13312套。
北京地区住房公积金个人贷款率（年末住房公积金个人贷款余额与年末住房公积金缴存余额的比率）为85.9%，比上年同期增加11.7个百分点。
自2015年1月1日起，个人贷款最高额度由80万元调整为120万元。按规定4次下调贷款利率，目前5年期以上贷款利率为3.25%。受委托办理住房公积金个人住房贷款业务的银行为中国工商银行、中国建设银行、中国农业银行、中国银行、交通银行、招商银行、中信银行和北京银行，共计8家，与上年相比无变化。
2.保障性住房建设项目贷款
按照住房城乡建设部的统一部署，根据市委、市政府工作要求，管理中心累计发放经适房、公租房、棚户区改造定向安置房项目贷款35个，建筑面积共855.18万平方米，涉及8.27万套住房。2015年，依据合同约定，按照工程进度，向11个项目发放贷款13.65亿元。当年应还贷款本金34.86亿元，实际回收贷款本金44.96亿元。截至2015年底，累计发放项目贷款181.02亿元，项目贷款余额102.60亿元。

（四）购买国债
2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。

（五）资金存储
截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。

（六）资金运用率
2015年住房